# Berlin Housing Market Analysis - Hauptanalyse
## Datenbereinigung, Zusammenführung und Explorative Analyse

### Projektübersicht
Dieses Notebook führt die Hauptanalyse des Berliner Wohnungsmarktes durch. Es baut auf dem PLZ-Mapping aus `01_Data_Preprocessing.ipynb` auf und erstellt eine umfassende Analyse der drei Datensätze.

### Voraussetzungen
- `01_Data_Preprocessing.ipynb` wurde erfolgreich ausgeführt
- PLZ-Mapping-Tabelle liegt in `data/processed/berlin_plz_mapping.csv` vor
- Originaldaten befinden sich in `data/raw/`

### Analyseziele
1. **Datenbereinigung und -normalisierung** aller drei Datensätze
2. **Zusammenführung** in ein einheitliches Dataset
3. **Zeitreihenanalyse** der Mietpreisentwicklung
4. **Bezirksvergleiche** mit statistischen Tests
5. **Explorative Datenanalyse** mit umfassenden Visualisierungen

---

**Autor**: Berlin Housing Market Analysis Team  
**Datum**: 4. Juli 2025  
**Version**: 1.0

## 1. Import Required Libraries
Importieren der erforderlichen Python-Bibliotheken für Datenverarbeitung, Analyse und Visualisierung.

In [2]:
# Datenmanipulation und -analyse
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualisierung
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Statistik und Machine Learning
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Textverarbeitung und Regex
import re
from collections import Counter

# Datum und Zeit
from datetime import datetime

# Konfiguration
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 50)
plt.style.use('seaborn-v0_8')

print("Alle Bibliotheken erfolgreich importiert!")
print(f"Pandas Version: {pd.__version__}")
print(f"NumPy Version: {np.__version__}")
print(f"Matplotlib Version: {matplotlib.__version__}")
print(f"Seaborn Version: {sns.__version__}")
print(f"Verarbeitung gestartet am: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Alle Bibliotheken erfolgreich importiert!
Pandas Version: 2.3.0
NumPy Version: 2.3.0
Matplotlib Version: 3.10.3
Seaborn Version: 0.13.2
Verarbeitung gestartet am: 2025-07-04 03:46:55


## 2. Laden der Originaldaten und PLZ-Mapping
### 2.1 PLZ-Mapping aus Preprocessing laden
Laden der PLZ-zu-Bezirk-Mapping-Tabelle die im vorherigen Notebook erstellt wurde.

In [3]:
# Lade PLZ-Mapping-Tabelle aus dem Preprocessing
print("=" * 60)
print("LADEN DER PLZ-MAPPING-TABELLE")
print("=" * 60)

try:
    plz_mapping_df = pd.read_csv('data/processed/berlin_plz_mapping.csv')
    print(f"PLZ-Mapping erfolgreich geladen: {len(plz_mapping_df)} PLZ-Zuordnungen")
    print(f"Abgedeckte Bezirke: {plz_mapping_df['Bezirk'].nunique()}")
    print(f"Bezirke: {sorted(plz_mapping_df['Bezirk'].unique())}")
except FileNotFoundError:
    print("FEHLER: PLZ-Mapping-Tabelle nicht gefunden!")
    print("Bitte führen Sie zuerst 01_Data_Preprocessing.ipynb aus.")
    raise

print("\n" + "=" * 60)
print("LADEN DER ORIGINALDATEN")
print("=" * 60)

# Lade alle drei Originaldatensätze
datasets = {}

# Dataset 2025 (Immobilienscout24)
print("\nDataset 2025 (Immobilienscout24)")
datasets['2025'] = pd.read_csv('data/raw/Dataset_2025.csv')
print(f"Zeilen: {datasets['2025'].shape[0]:,}")
print(f"Spalten: {datasets['2025'].shape[1]}")
print(f"Spalten: {list(datasets['2025'].columns)}")

# Dataset 2018-2019 (Kaggle)
print("\nDataset 2018-2019 (Kaggle)")
datasets['2018_2019'] = pd.read_csv('data/raw/Dataset_2018_2019.csv')
print(f"Zeilen: {datasets['2018_2019'].shape[0]:,}")
print(f"Spalten: {datasets['2018_2019'].shape[1]}")
print(f"Spalten: {list(datasets['2018_2019'].columns)}")

# Dataset 2022 (Springer/Immowelt/Immonet)
print("\nDataset 2022 (Springer/Immowelt/Immonet)")
datasets['2022'] = pd.read_csv('data/raw/Dataset_2022.csv')
print(f"Zeilen: {datasets['2022'].shape[0]:,}")
print(f"Spalten: {datasets['2022'].shape[1]}")
print(f"Erste 10 Spalten: {list(datasets['2022'].columns[:10])}")

total_rows = sum(df.shape[0] for df in datasets.values())
print(f"\nGesamtanzahl Rohdaten: {total_rows:,} Datenpunkte aus {len(datasets)} Quellen")

LADEN DER PLZ-MAPPING-TABELLE
PLZ-Mapping erfolgreich geladen: 208 PLZ-Zuordnungen
Abgedeckte Bezirke: 12
Bezirke: ['Charlottenburg-Wilmersdorf', 'Friedrichshain-Kreuzberg', 'Lichtenberg', 'Marzahn-Hellersdorf', 'Mitte', 'Neukölln', 'Pankow', 'Reinickendorf', 'Spandau', 'Steglitz-Zehlendorf', 'Tempelhof-Schöneberg', 'Treptow-Köpenick']

LADEN DER ORIGINALDATEN

Dataset 2025 (Immobilienscout24)
Zeilen: 6,109
Spalten: 5
Spalten: ['title', 'price', 'size', 'address', 'link']

Dataset 2018-2019 (Kaggle)
Zeilen: 10,406
Spalten: 9
Spalten: ['regio3', 'street', 'livingSpace', 'baseRent', 'totalRent', 'noRooms', 'floor', 'typeOfFlat', 'yearConstructed']

Dataset 2022 (Springer/Immowelt/Immonet)
Zeilen: 2,950
Spalten: 75
Erste 10 Spalten: ['ID', 'SORTE', 'PLZ', 'KALTMIETE', 'WARMMIETE', 'NEBENKOSTEN', 'KAUTION', 'HEIZUNGSKOSTEN', 'ZIMMER', 'PARKPLAETZE']

Gesamtanzahl Rohdaten: 19,465 Datenpunkte aus 3 Quellen


### 2.2 Erste Datenqualitätsanalyse
Analysiere die Struktur und Qualität der drei Datensätze vor der Bereinigung.

In [ ]:
# Detaillierte Analyse jedes Datensatzes
print("=" * 80)
print("DATENQUALITÄTSANALYSE")
print("=" * 80)

for year, df in datasets.items():
    print(f"\n{'='*20} DATASET {year} {'='*20}")
    print(f"Shape: {df.shape}")
    print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Fehlende Werte
    missing_info = df.isnull().sum()
    missing_pct = (missing_info / len(df) * 100).round(2)
    
    if missing_info.sum() > 0:
        print(f"\nFehlende Werte:")
        for col in missing_info[missing_info > 0].index:
            print(f"  {col}: {missing_info[col]} ({missing_pct[col]}%)")
    else:
        print("\nKeine fehlenden Werte gefunden")
    
    # Datentypen
    print(f"\nDatentypen:")
    print(df.dtypes.value_counts())
    
    # Erste 3 Zeilen als Beispiel
    print(f"\nBeispieldaten (erste 3 Zeilen):")
    print(df.head(3))
    
    print(f"\nNumerische Spalten Statistik:")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(df[numeric_cols].describe())
    else:
        print("Keine numerischen Spalten gefunden")

print(f"\n{'='*80}")
print("ZUSAMMENFASSUNG DER DATENQUALITÄT")
print(f"{'='*80}")
print(f"Dataset 2025: {datasets['2025'].shape[0]:,} Zeilen, {datasets['2025'].shape[1]} Spalten")
print(f"Dataset 2018-2019: {datasets['2018_2019'].shape[0]:,} Zeilen, {datasets['2018_2019'].shape[1]} Spalten") 
print(f"Dataset 2022: {datasets['2022'].shape[0]:,} Zeilen, {datasets['2022'].shape[1]} Spalten")
print(f"Gesamtdatenpunkte: {sum(df.shape[0] for df in datasets.values()):,}")

## 3. Datenbereinigung und Normalisierung
### 3.1 Dataset 2022: PLZ-zu-Bezirk-Zuordnung anwenden

Das kritische PLZ-Problem aus Phase 1 wird jetzt gelöst, indem wir die erstellte Mapping-Tabelle anwenden.

In [ ]:
# PLZ-zu-Bezirk-Zuordnung für Dataset 2022
print("=" * 60)
print("PLZ-ZU-BEZIRK-ZUORDNUNG FÜR DATASET 2022")
print("=" * 60)

# Erstelle Kopie für Bearbeitung
df_2022 = datasets['2022'].copy()

# PLZ als String für Matching
df_2022['PLZ_str'] = df_2022['PLZ'].astype(str)

# Merge mit PLZ-Mapping
df_2022_mapped = df_2022.merge(
    plz_mapping_df, 
    left_on='PLZ_str', 
    right_on='PLZ', 
    how='left'
)

# Statistik der Zuordnung
total_entries = len(df_2022_mapped)
successful_mappings = df_2022_mapped['Bezirk'].notna().sum()
mapping_rate = (successful_mappings / total_entries) * 100

print(f"Gesamteinträge Dataset 2022: {total_entries:,}")
print(f"Erfolgreich zugeordnet: {successful_mappings:,}")
print(f"Zuordnungsrate: {mapping_rate:.1f}%")

# Nicht zugeordnete PLZ analysieren
unmapped = df_2022_mapped[df_2022_mapped['Bezirk'].isna()]
if len(unmapped) > 0:
    print(f"\nNicht zugeordnete PLZ ({len(unmapped)} Einträge):")
    unmapped_plz_counts = unmapped['PLZ_str'].value_counts()
    for plz, count in unmapped_plz_counts.items():
        print(f"  PLZ {plz}: {count} Einträge")

# Bezirksverteilung
print(f"\nBezirksverteilung (Top 10):")
bezirk_counts = df_2022_mapped['Bezirk'].value_counts()
for bezirk, count in bezirk_counts.head(10).items():
    print(f"  {bezirk}: {count} Einträge")

# Nur erfolgreich zugeordnete Daten behalten
df_2022_clean = df_2022_mapped[df_2022_mapped['Bezirk'].notna()].copy()
print(f"\nBereinigte Dataset 2022: {len(df_2022_clean):,} Einträge (Verlust: {len(df_2022) - len(df_2022_clean)} Einträge)")